In [6]:
import numpy as np
from sys import path
path.append('../tools')
from tools import logsig

In [7]:
def get_expected_output(X):
    return logsig(np.dot(logsig(X * np.array([10, 10]) + np.array([-5, 5])), np.array([1, 1])) + -1)

In [8]:
vec_get_y = np.vectorize(get_expected_output)

In [9]:
x_train = np.linspace(-2, 2, 82)
y_train = vec_get_y(x_train)

In [10]:
x_train = x_train.reshape(len(x_train), 1)
y_train = y_train.reshape(len(y_train), 1)

In [3]:
# Create the model with keras and tensorflow
# Adapted from : https://keras.io/guides/custom_train_step_in_tensorflow/

# Imports. Need to do this environment stuff for some reason
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import tensorflow as tf
import keras
from keras import layers

In [12]:
class CustomModel(keras.Model):
    @tf.function
    def train_step(self, data):
        # Unpack data
        x, y = data

        # Compute the loss
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)
            loss = self.compute_loss(y=y, y_pred=y_pred)

        # Compute the gradient
        gradient = tape.gradient(loss, self.trainable_variables)

        # Update the weights
        self.optimizer.apply(gradient, self.trainable_variables)

        # Update metrics
        for metric in self.metrics:
            if metric.name == 'loss':
                metric.update_state(loss)
            else:
                metric.update_state(y, y_pred)

        # Return the metrics
        return {m.name: m.result() for m in self.metrics}

In [13]:
# Construct and compile an instance of CustomModel
inputs = keras.Input(shape=(1,))
x = keras.layers.Dense(2)(inputs)
outputs = keras.layers.Dense(1)(x)
model = CustomModel(inputs, outputs)
model.compile(optimizer="SGD", loss="mse", metrics=['accuracy', 'mse'])

model.fit(x_train, y_train, epochs=3)

Epoch 1/3
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - mse: 2.7979 - loss: 2.7685  
Epoch 2/3
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 943us/step - accuracy: 0.0000e+00 - mse: 1.7943 - loss: 1.7864
Epoch 3/3
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 952us/step - accuracy: 0.0000e+00 - mse: 1.2364 - loss: 1.2440


In [16]:
y_pred = model(x_train, training=False)